In [1]:
# parameters.ipynb

import geopandas as gdp
import numpy as np
import pandas as pd
import itertools
from scenarios import scenarios  # Import scenario configurations
from scipy.spatial.distance import cdist

In [3]:
# Select Scenario
scenario_name = "nguenyyiel_terkidi_baseline"  # Change this to switch scenarios
params = scenarios[scenario_name]  # Load selected scenario parameters

In [4]:
# Load the GeoJSON file
location_nodes =  gdp.read_file(params["location_file"])
location_nodes

,Name,Camp,type_f,id,geometry
0,Nguenyyiel1,Nguenyyiel,demand_point,None,POINT (34.32649 8.31716)
1,Nguenyyiel2,Nguenyyiel,demand_point,None,POINT (34.32866 8.2964)
2,Nguenyyiel3,Nguenyyiel,demand_point,None,POINT (34.31006 8.32614)
3,Nguenyyiel4,Nguenyyiel,demand_point,None,POINT (34.31804 8.31556)
4,Nguenyyiel5,Nguenyyiel,demand_point,None,POINT (34.33447 8.29181)
...,...,...,...,...,...
329,HC_CandidateLocation3,Tierkidi,HC,hc3,POINT (34.28141 8.27974)
330,HC_CandidateLocation4,Tierkidi,HC,hc4,POINT (34.28596 8.2732)
331,HC_CandidateLocation5,Tierkidi,HC,hc5,POINT (34.29226 8.2789)
332,HC_CandidateLocation6,Tierkidi,HC,hc6,POINT (34.27802 8.27741)


In [5]:
# Add x and y coordinates
location_nodes.loc[:, 'x'] = location_nodes.geometry.x
location_nodes.loc[:, 'y'] = location_nodes.geometry.y

# Sort 
location_nodes = location_nodes.sort_values(by=['y', 'x']).reset_index(drop=True)

# Label sorted demand points
demand_points_gdf = location_nodes.loc[location_nodes.type_f == "demand_point"].copy()
demand_points_gdf['label'] = ['i' + str(i + 1) for i in range(len(demand_points_gdf))]

# Save demand point labels to a Numpy Array
dps = demand_points_gdf['label'].to_numpy()

# Subset location types
hps_gdf = location_nodes[location_nodes.type_f == "HP"]
hcs_gdf = location_nodes[location_nodes.type_f == "HC"]
hfs_gdf = location_nodes[(location_nodes.type_f == "HC") | (location_nodes.type_f == "HP")].drop_duplicates(subset='geometry').reset_index(drop=False)

# Label candidate locations
hfs_gdf['label'] = ['j' + str(j + 1) for j in range(len(hfs_gdf))]

# Save location labels
hfs = hfs_gdf['label'].to_numpy()
hps = hfs_gdf[hfs_gdf['geometry'].isin(hps_gdf['geometry'])]['label'].to_numpy()
hcs = hfs_gdf[hfs_gdf['geometry'].isin(hcs_gdf['geometry'])]['label'].to_numpy()


In [9]:
hcs

array(['j2', 'j3', 'j5', 'j7', 'j9', 'j11', 'j14', 'j17', 'j20', 'j21',
       'j22', 'j24', 'j28', 'j29'], dtype=object)

In [10]:
camps = set(location_nodes["Camp"].unique())
camps
camp_demand_labels = demand_points_gdf.groupby("Camp")["label"].apply(set).to_dict()
camp_demand_labels
camp_candidate_location_labels = hfs_gdf.groupby("Camp")["label"].apply(set).to_dict()
camp_candidate_location_labels 

{'Nguenyyiel': {'j16',
  'j17',
  'j18',
  'j19',
  'j20',
  'j21',
  'j22',
  'j23',
  'j24',
  'j25',
  'j26',
  'j27',
  'j28',
  'j29',
  'j30',
  'j31',
  'j32'},
 'Tierkidi': {'j1',
  'j10',
  'j11',
  'j12',
  'j13',
  'j14',
  'j15',
  'j2',
  'j3',
  'j4',
  'j5',
  'j6',
  'j7',
  'j8',
  'j9'}}

In [12]:
# This is _just_ to have insights on which values would be adequate for t'max and t''max 


# Initialize dictionaries to store results
avg_dist_demand_to_hp = {}
avg_dist_demand_to_hc = {}
min_intercamp_distance = {} # For distances between different camps
max_withincamp_distance = {} # For maximum distance within the same camp

# Compute distances for each camp
for camp in camps:
    # Filter locations by camp
    camp_demand_points = demand_points_gdf[demand_points_gdf["Camp"] == camp]
    camp_hps = hps_gdf[hps_gdf["Camp"] == camp]
    camp_hcs = hcs_gdf[hcs_gdf["Camp"] == camp]

    # Compute distances between demand points and HPs
    if not camp_hps.empty:
        avg_distances_per_demand_point_to_hp = camp_demand_points.to_crs(epsg=3857).geometry.apply(lambda dp: camp_hps.to_crs(epsg=3857).geometry.distance(dp).mean())
        avg_dist_demand_to_hp[camp] = avg_distances_per_demand_point_to_hp.mean()
    else:
        avg_dist_demand_to_hp[camp] = None  # No HPs in this camp

    # Compute distances between demand points and HCs
    if not camp_hcs.empty:
        avg_distances_per_demand_point_to_hc = camp_demand_points.to_crs(epsg=3857).geometry.apply(lambda dp: camp_hcs.to_crs(epsg=3857).geometry.distance(dp).mean())
        avg_dist_demand_to_hc[camp] = avg_distances_per_demand_point_to_hc.mean()
    else:
        avg_dist_demand_to_hc[camp] = None  # No HCs in this camp

    # Now calculate maximum within-camp distance (distance between any two locations within the same camp)
    locations_camp = location_nodes[location_nodes["Camp"] == camp]
    
    # Reproject to EPSG:3857 (meters) and compute the maximum pairwise distance within the camp
    max_within_distance = locations_camp.to_crs(epsg=3857).geometry.apply(
        lambda loc: locations_camp.to_crs(epsg=3857).geometry.distance(loc).max()
    ).max()
    
    # Store the maximum distance within the camp
    max_withincamp_distance[camp] = max_within_distance


# Compute minimum distance between any location in different camps
if len(camps)>1:
    for camp1 in camps:
        for camp2 in camps:
            if camp1 != camp2:
                locations_camp1 = location_nodes[location_nodes["Camp"] == camp1]
                locations_camp2 = location_nodes[location_nodes["Camp"] == camp2]
                min_distance = locations_camp1.to_crs(epsg=3857).geometry.apply(lambda loc: locations_camp2.to_crs(epsg=3857).geometry.distance(loc).min()).min()
                min_intercamp_distance[(camp1, camp2)] = min_distance

    # Now find the overall minimum distance across all inter-camp pairs
    overall_min_distance = min(min_intercamp_distance.values())

# Find the overall maximum within-camp distance
overall_max_withincamp_distance = max(max_withincamp_distance.values())


# # Convert results to DataFrames for better visualization
# df_avg_dist_hp = pd.DataFrame(list(avg_dist_demand_to_hp.items()), columns=["Camp", "Avg_Dist_Demand_to_HP"])
# df_avg_dist_hc = pd.DataFrame(list(avg_dist_demand_to_hc.items()), columns=["Camp", "Avg_Dist_Demand_to_HC"])
# df_min_intercamp = pd.DataFrame(list(overall_min_distance.items()), columns=["Camp_Pair", "Min_Distance"])
# df_max_withincamp = pd.DataFrame(list(overall_max_withincamp_distance.items()), columns=["Camp", "Max_Distance"])

# Display results
print(avg_dist_demand_to_hp)
print(avg_dist_demand_to_hc)
if len(camps)>1: print(min_intercamp_distance)
if len(camps)>1: print(overall_min_distance)
print(max_withincamp_distance)
print(overall_max_withincamp_distance)



{'Tierkidi': np.float64(1277.9341789243774), 'Nguenyyiel': np.float64(2280.564951867882)}
{'Tierkidi': np.float64(1317.5746385943735), 'Nguenyyiel': np.float64(2121.9905892858524)}
{('Tierkidi', 'Nguenyyiel'): np.float64(3276.4746712079254), ('Nguenyyiel', 'Tierkidi'): np.float64(3276.4746712079254)}
3276.4746712079254
{'Tierkidi': np.float64(3844.636087954079), 'Nguenyyiel': np.float64(5884.276762274655)}
5884.276762274655


In [13]:
def compute_distance_matrix(demand_points_gdf, hfs_gdf):
    """
    Compute the distance matrix between demand points and candidate health facility locations.

    Parameters:
    - demand_points_gdf: GeoDataFrame containing demand points with 'geometry'.
    - hfs_gdf: GeoDataFrame containing candidate health facility locations with 'geometry'.

    Returns:
    - distance_matrix: 2D NumPy array of distances (rows: demand points, columns: health facilities).
    """
    # Extract coordinates as NumPy arrays directly from the geometry column
    demand_coords = np.array(demand_points_gdf.geometry.apply(lambda point: (point.x, point.y)).tolist())
    hfs_coords = np.array(hfs_gdf.geometry.apply(lambda point: (point.x, point.y)).tolist())
    
    # Compute the distance matrix using cdist with Euclidean metric
    distance_matrix = cdist(demand_coords, hfs_coords, metric='euclidean')
    
    # Create a labeled DataFrame
    distance_df = pd.DataFrame(distance_matrix, index=dps, columns=hfs)

    return distance_df

In [14]:
# Example usage:
distance_df = compute_distance_matrix(demand_points_gdf, hfs_gdf)

# To save the above matrix into an Excel file to subsequently read
# distance_df.to_excel('distance_matrix_refcamps.xlsx', sheet_name='DistanceMatrixRefCamps')#, float_format="%.2f")

# Distance matrix
# distance_matrix = pd.read_excel('distance_matrix_ij.xlsx', index_col=0)
# distance_matrix = pd.read_excel('distance_matrix_refcamps.xlsx', index_col=0)
distance_df

,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,...,j23,j24,j25,j26,j27,j28,j29,j30,j31,j32
i1,0.003845,0.004302,0.014429,0.006830,0.009975,0.008775,0.019582,0.012138,0.012716,0.014477,...,0.053120,0.057903,0.058556,0.060235,0.059649,0.062475,0.060620,0.059941,0.063855,0.062294
i2,0.002556,0.003325,0.013120,0.006096,0.010487,0.009098,0.020418,0.012655,0.011423,0.013193,...,0.053402,0.058120,0.058777,0.060368,0.059566,0.062489,0.060395,0.059569,0.063709,0.061992
i3,0.002928,0.003065,0.013741,0.005562,0.009444,0.008067,0.019410,0.011612,0.011710,0.013444,...,0.052387,0.057119,0.057775,0.059388,0.058658,0.061545,0.059548,0.058797,0.062827,0.061183
i4,0.001768,0.002437,0.012538,0.005263,0.010309,0.008804,0.020450,0.012464,0.010626,0.012382,...,0.052998,0.057675,0.058335,0.059872,0.058953,0.061927,0.059713,0.058823,0.063064,0.061275
i5,0.001132,0.001838,0.011973,0.004738,0.010408,0.008817,0.020684,0.012545,0.009936,0.011685,...,0.052865,0.057506,0.058167,0.059657,0.058630,0.061650,0.059322,0.058365,0.062709,0.060847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i296,0.061725,0.059958,0.063807,0.057058,0.054704,0.054968,0.052352,0.053079,0.059306,0.058554,...,0.025066,0.021756,0.021921,0.017879,0.008315,0.012406,0.002626,0.006799,0.004921,0.002843
i297,0.061273,0.059535,0.062876,0.056621,0.054695,0.054859,0.052901,0.053150,0.058458,0.057630,...,0.027300,0.024286,0.024490,0.020545,0.010815,0.015209,0.004370,0.004293,0.007807,0.000484
i298,0.063800,0.062005,0.066410,0.059130,0.056270,0.056658,0.053206,0.054553,0.061829,0.061161,...,0.023410,0.019468,0.019529,0.015279,0.006819,0.009444,0.004935,0.010753,0.001781,0.006775
i299,0.061983,0.060230,0.063810,0.057322,0.055184,0.055398,0.053097,0.053597,0.059353,0.058559,...,0.026472,0.023237,0.023410,0.019381,0.009778,0.013910,0.003692,0.005778,0.006360,0.001694


In [15]:
def compute_distance_matrix_meters(demand_points_gdf, hfs_gdf, crs_epsg=3857):
    """
    Compute the distance matrix between demand points and candidate health facility locations.

    Parameters:
    - demand_points_gdf: GeoDataFrame containing the demand points with geometry (usually point geometries).
    - hfs_gdf: GeoDataFrame containing the candidate health facility locations with geometry.
    - crs_epsg: The EPSG code to which the geometries will be reprojected. Default is 3857 (Web Mercator).

    Returns:
    - distance_df: A pandas DataFrame where the rows are demand points, the columns are health facilities,
                   and the values are the distances between them.
    """
    # Reproject both demand points and health facilities to the target CRS (e.g., EPSG:3857 for meters)
    demand_points_gdf = demand_points_gdf.to_crs(epsg=crs_epsg)
    hfs_gdf = hfs_gdf.to_crs(epsg=crs_epsg)

    # Initialize an empty distance matrix with dimensions (num_demand_points x num_health_facilities)
    num_demand_points = len(demand_points_gdf)
    num_health_facilities = len(hfs_gdf)
    distance_matrix = np.zeros((num_demand_points, num_health_facilities))

    # Compute distances
    for i, demand_point in enumerate(demand_points_gdf.geometry):
        for j, hf_location in enumerate(hfs_gdf.geometry):
            distance_matrix[i, j] = demand_point.distance(hf_location)
    
    # Create a DataFrame with labeled indices and columns
    distance_df = pd.DataFrame(distance_matrix, index=dps, columns=hfs) 


    return distance_df


In [18]:
# Example usage:
distance_df = compute_distance_matrix_meters(demand_points_gdf, hfs_gdf)
distance_df


,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,...,j23,j24,j25,j26,j27,j28,j29,j30,j31,j32
i1,429.747320,483.009236,1608.217048,768.056994,1115.820708,983.222261,2183.725454,1357.518175,1421.591434,1618.935268,...,5950.322894,6489.364848,6562.302132,6755.013125,6698.900460,7012.007926,6812.990566,6740.632405,7173.837062,7003.551824
i2,285.970942,373.820129,1462.200515,685.722922,1171.629937,1017.917183,2276.082289,1413.890451,1277.333958,1475.681329,...,5980.400202,6512.259852,6585.660245,6768.496713,6688.419564,7012.290593,6786.824780,6698.174393,7156.350089,6968.767847
i3,326.514256,343.904184,1530.887816,625.518144,1055.129051,902.694226,2163.514305,1297.392835,1308.270437,1502.591511,...,5867.044848,6400.410796,6473.710610,6659.003771,6586.882654,6906.844182,6692.051957,6611.771864,7057.703906,6878.257330
i4,197.414829,274.043169,1396.948516,592.028918,1150.754873,983.967925,2278.915856,1391.662286,1187.738681,1384.509006,...,5934.220704,6461.505180,6535.168690,6712.113474,6619.038794,6948.486644,6709.824171,6614.096529,7083.405630,6887.886737
i5,126.121647,206.742237,1333.704432,532.821316,1161.182438,984.652418,2304.599752,1399.943846,1110.434293,1306.436839,...,5918.542925,6441.722067,6515.607042,6687.259995,6582.088502,6916.784629,6665.355747,6562.265619,7042.941196,6839.419497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i296,6935.401424,6737.305117,7159.695643,6411.222131,6151.591358,6180.397994,5889.586642,5969.542078,6656.298624,6570.310950,...,2808.377263,2432.678040,2450.396864,1996.880100,930.510186,1383.673746,295.464343,757.325866,547.950687,316.471905
i297,6886.708770,6691.739794,7057.891536,6364.142429,6151.801247,6169.435731,5951.124595,5978.487987,6563.769043,6469.311450,...,3056.438960,2713.848697,2735.883915,2293.420390,1208.281837,1695.600237,489.115482,479.044584,869.329354,54.481266
i298,7165.812830,6964.626436,7448.616070,6641.264370,6325.805047,6368.264636,5985.191608,6133.560233,6936.416371,6859.670218,...,2626.595987,2180.408648,2186.505018,1709.680671,766.399792,1055.800011,552.016176,1197.583519,199.729401,754.270016
i299,6965.481950,6768.949539,7161.539897,6441.987840,6206.155651,6229.431963,5973.347083,6028.330887,6663.072788,6572.449142,...,2965.149493,2597.949039,2616.514884,2164.567136,1093.729795,1551.490412,414.591357,644.281252,708.475577,188.938540


In [22]:

# To save the above matrix into an Excel file to subsequently read
# distance_df.to_excel('distance_matrix_refcamps_meters.xlsx', sheet_name='DistanceMatrixRefCamps')#, float_format="%.2f")
# distance_df.to_excel('distance_matrix_tierkidi.xlsx', sheet_name='DistanceMatrixTierkidi')#, float_format="%.2f")
# distance_df.to_excel('distance_matrix_terkidi_baseline.xlsx', sheet_name='DistanceMatrixTierkidiBaseline')#, float_format="%.2f")
# distance_df.to_excel('distance_matrix_nguenyyiel_baseline.xlsx', sheet_name='DistMatrixNguenyyielBaseline')#, float_format="%.2f")
# distance_df.to_excel('distance_matrix_kule_baseline.xlsx', sheet_name='DistMatrixKuleBaseline')#, float_format="%.2f")
# distance_df.to_excel('distance_matrix_jewi_baseline.xlsx', sheet_name='DistMatrixJewiBaseline')#, float_format="%.2f")
# distance_df.to_excel('distance_matrix_pugnido_baseline.xlsx', sheet_name='DistMatrixPugnidoBaseline')#, float_format="%.2f")
# distance_df.to_excel('distance_matrix_nguenyyiel_terkidi_baseline.xlsx', sheet_name='DistMatrixNgTerkBaseline')#, float_format="%.2f")

# Distance matrix
# distance_matrix = pd.read_excel('distance_matrix_ij.xlsx', index_col=0)
# distance_matrix = pd.read_excel('distance_matrix_refcamps_meters.xlsx', index_col=0)
distance_matrix = pd.read_excel('individual_refugee_camps/distance_matrix_nguenyyiel_terkidi_baseline.xlsx', index_col=0)
distance_matrix

,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,...,j23,j24,j25,j26,j27,j28,j29,j30,j31,j32
i1,429.747320,483.009236,1608.217048,768.056994,1115.820708,983.222261,2183.725454,1357.518175,1421.591434,1618.935268,...,5950.322894,6489.364848,6562.302132,6755.013125,6698.900460,7012.007926,6812.990566,6740.632405,7173.837062,7003.551824
i2,285.970942,373.820129,1462.200515,685.722922,1171.629937,1017.917183,2276.082289,1413.890451,1277.333958,1475.681329,...,5980.400202,6512.259852,6585.660245,6768.496713,6688.419564,7012.290593,6786.824780,6698.174393,7156.350089,6968.767847
i3,326.514256,343.904184,1530.887816,625.518144,1055.129051,902.694226,2163.514305,1297.392835,1308.270437,1502.591511,...,5867.044848,6400.410796,6473.710610,6659.003771,6586.882654,6906.844182,6692.051957,6611.771864,7057.703906,6878.257330
i4,197.414829,274.043169,1396.948516,592.028918,1150.754873,983.967925,2278.915856,1391.662286,1187.738681,1384.509006,...,5934.220704,6461.505180,6535.168690,6712.113474,6619.038794,6948.486644,6709.824171,6614.096529,7083.405630,6887.886737
i5,126.121647,206.742237,1333.704432,532.821316,1161.182438,984.652418,2304.599752,1399.943846,1110.434293,1306.436839,...,5918.542925,6441.722067,6515.607042,6687.259995,6582.088502,6916.784629,6665.355747,6562.265619,7042.941196,6839.419497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i296,6935.401424,6737.305117,7159.695643,6411.222131,6151.591358,6180.397994,5889.586642,5969.542078,6656.298624,6570.310950,...,2808.377263,2432.678040,2450.396864,1996.880100,930.510186,1383.673746,295.464343,757.325866,547.950687,316.471905
i297,6886.708770,6691.739794,7057.891536,6364.142429,6151.801247,6169.435731,5951.124595,5978.487987,6563.769043,6469.311450,...,3056.438960,2713.848697,2735.883915,2293.420390,1208.281837,1695.600237,489.115482,479.044584,869.329354,54.481266
i298,7165.812830,6964.626436,7448.616070,6641.264370,6325.805047,6368.264636,5985.191608,6133.560233,6936.416371,6859.670218,...,2626.595987,2180.408648,2186.505018,1709.680671,766.399792,1055.800011,552.016176,1197.583519,199.729401,754.270016
i299,6965.481950,6768.949539,7161.539897,6441.987840,6206.155651,6229.431963,5973.347083,6028.330887,6663.072788,6572.449142,...,2965.149493,2597.949039,2616.514884,2164.567136,1093.729795,1551.490412,414.591357,644.281252,708.475577,188.938540


In [23]:
######################################

# Health services and workers
services = ['basic','maternal1','maternal2']
health_workers = ['doctor','nurse','midwife']
levels = ['hp', 'hc']



# Assign scenario parameters
HFs_to_locate = params["HFs_to_locate"]
t1max = params["t1max"]
t2max = params["t2max"]
workers_to_allocate = params["workers_to_allocate"]
working_hours = params["working_hours"]
service_time = params["service_time"]

# Lower bound workers per HF type
lb_workers_df = pd.DataFrame(params["lb_workers"], index=health_workers)
lb_workers = {(health_workers[p], levels[l]): lb_workers_df.iloc[p, l] 
      for p, l in itertools.product(range(len(health_workers)), range(len(levels)))}

# Upper bound workers per HF type
ub_workers_df = pd.DataFrame(params["ub_workers"], index=health_workers)
ub_workers = {(health_workers[p], levels[l]): ub_workers_df.iloc[p, l] 
      for p, l in itertools.product(range(len(health_workers)), range(len(levels)))}

# Where can each service be provided?
services_at_HFs_df = pd.DataFrame(params["services_at_HFs"], index=services)
a_HF = {(services[s], levels[l]): services_at_HFs_df.iloc[s, l] 
      for s, l in itertools.product(range(len(services)), range(len(levels)))}

# Which health worker can deliver each service?
services_per_worker_df = pd.DataFrame(params["services_per_worker"], index=health_workers)
a_W = {(health_workers[p], services[s]): services_per_worker_df.iloc[p, s] 
      for p, s in itertools.product(range(len(health_workers)), range(len(services)))}

# Demand rates
total_population = {(key): params["total_population"] for key in dps}

# Opening hours
demand_rate_opening_hours_df = pd.DataFrame([params["demand_rate_opening_hours"]] * len(dps), index=dps, columns=services)
dr_oh = {(dps[i], services[s]): demand_rate_opening_hours_df.iloc[i, s] 
      for i, s in itertools.product(range(len(dps)), range(len(services)))}

dd_oh = {(key): int(round(total_population[i] * dr_oh[key])) for i in dps for key in dr_oh}

# Closing hours
demand_rate_closing_hours_df = pd.DataFrame([params["demand_rate_closing_hours"]] * len(dps), index=dps, columns=services)
dr_ch = {(dps[i], services[s]): demand_rate_closing_hours_df.iloc[i, s] 
      for i, s in itertools.product(range(len(dps)), range(len(services)))}

dd_ch = {(key): int(round(total_population[i] * dr_ch[key])) for i in dps for key in dr_ch}


In [24]:
# Define the sets and parameters to use in the model
I = dps
J = hfs
J_HP = hps
J_HC = hcs
C = np.array(list(camps), dtype=object)
I_c = {key: np.array(list(value), dtype=object) for key, value in camp_demand_labels.items()} #check
J_c = {key: np.array(list(value), dtype=object) for key, value in camp_candidate_location_labels.items()} #check
t = distance_matrix # travel time (distances) between demand points and HFs
S = services
P = health_workers
L = levels
n_HF = dict(zip(levels, HFs_to_locate))
n_W = dict(zip(health_workers, workers_to_allocate))
Pi = total_population #new Feb 19
r1 = dr_oh #new Feb 19
r2 = dr_ch #new Feb 19
d1 = dd_oh #new Feb 19; either this is necessary, or Pi and r1
d2 = dd_ch #new Feb 19; either this is necessary, or Pi and r2
lb = lb_workers
ub = ub_workers
q = dict(zip(services, service_time))
h = dict(zip(health_workers, working_hours))


model_data = {
    "I": I,
    "J": J,
    "J_HP": J_HP,
    "J_HC": J_HC,
    "C": C,
    "I_c": I_c,
    "J_c": J_c,
    "S": S,
    "P": P,
    "L": L,
    "t": t,
    "t1max": t1max,
    "t2max": t2max,
    "n_HF": n_HF,
    "n_W": n_W,
    "a_HF": a_HF,
    "a_W": a_W,
    "Pi": Pi,
    "r1": r1,
    "r2": r2,
    "d1": d1,
    "d2": d2,
    "lb": lb,
    "ub": ub,
    "q": q,
    "h": h
}